In [86]:
import numpy as np

import lightgbm as lgb

import scipy.sparse

from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss

from bayes_opt import BayesianOptimization

In [87]:
# Generate synthetic dataset

X,y = make_classification(n_samples=5000, n_features=20, random_state=42)

In [88]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)

In [89]:
# Convert to LightGBM dataset


train_data = lgb.Dataset(X_train, label=y_train)

test_data = lgb.Dataset(X_test, label=y_test)

1. Custom Loss Functions in LightGBM

By default, LightGBM uses predefined loss functions like binary log loss or MSE.
But we can define our own custom loss function to guide optimization.


define a custom log loss function and its gradient & hessian for LightGBM.

In [90]:
# Custom Log Loss function (for binary classification)

def custom_log_loss(y_true,y_pred):

    eps = 1e-15

    y_pred = np.clip(y_pred, eps, 1-eps)

    # Gradient (first derivative)

    grad = y_pred - y_true

    # Hessian (second derivative)

    hess = y_pred * (1 - y_pred)

    return grad,hess

In [91]:
# Train model with custom loss
params = {


    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'num_leaves': 31

    
}

In [92]:
X_train = X_train.astype(np.float32)

X_test = X_test.astype(np.float32)

In [93]:
model = lgb.train(
    params, 
    train_data, 
    valid_sets=[test_data], 
    num_boost_round=200,
    callbacks=[
        lgb.early_stopping(stopping_rounds=30),  
        lgb.log_evaluation(period=50)  
    ]
    )

Training until validation scores don't improve for 30 rounds
[50]	valid_0's binary_logloss: 0.217242
[100]	valid_0's binary_logloss: 0.200936
Early stopping, best iteration is:
[76]	valid_0's binary_logloss: 0.20056


In [94]:
# Predictions

y_pred = model.predict(X_test)

In [95]:
loss = log_loss(y_test, y_pred)

print(f"Custom Log Loss: {loss:.4f}")

Custom Log Loss: 0.2006


 2. Bayesian Optimization for Hyperparameter Tuning
 
 
 Instead of grid search or random search, we use Bayesian Optimization for smarter tuning.
It learns from past trials and selects better hyperparameters efficiently.

In [96]:
# Define objective function

def lgb_evaluate(num_leaves, learning_rate, max_depth):

    hyper_params = {
        
        'objective' : 'binary',
        'metric'    : 'binary_error',
        'boosting_type':'gbdt',
        'num_leaves' : int(num_leaves),
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'verbose': -1
    }

    hyper_model = lgb.train(
    hyper_params, 
    train_data, 
    valid_sets=[test_data], 
    num_boost_round=100, 
    callbacks=[
        lgb.early_stopping(stopping_rounds=10),  
        lgb.log_evaluation(period=False)  
    ]
    )

    hyper_preds = hyper_model.predict(X_test)

    hyper_accuracy = np.mean(( hyper_preds > 0.5) == y_test)

    return hyper_accuracy


In [97]:
# Define search space

optimizer = BayesianOptimization(

      f=lgb_evaluate,
      pbounds={
         'num_leaves': (10,100),
         'learning_rate':(0.01,0.3),
         'max_depth': (3,12)

       },
    random_state=42
)


In [98]:
# Run optimization

optimizer.maximize(init_points=5, n_iter=10)

|   iter    |  target   | learni... | max_depth | num_le... |
-------------------------------------------------------------
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[11]	valid_0's binary_error: 0.07
| 1         | 0.93      | 0.1186    | 11.56     | 75.88     |
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's binary_error: 0.079
| 2         | 0.921     | 0.1836    | 4.404     | 24.04     |
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	valid_0's binary_error: 0.07
| 3         | 0.93      | 0.02684   | 10.8      | 64.1      |
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	valid_0's binary_error: 0.083
| 4         | 0.917     | 0.2153    | 3.185     | 97.29     |
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[18]	valid_

In [99]:
# Print best parameters

print("Best Parameters :", optimizer.max)

Best Parameters : {'target': 0.93, 'params': {'learning_rate': 0.11861663446573512, 'max_depth': 11.556428757689245, 'num_leaves': 75.87945476302646}}


LightGBM allows monotonic constraints, ensuring that a feature’s impact is always increasing or decreasing.
Example: Ensuring Price Always Increases with Age

✅ Why Monotonic Constraints?

Prevent counterintuitive results (e.g., higher salary should not decrease approval rate)
Adds trust to the model for business applications

In [ ]:
mono_params = {

    'objective': 'regression',
    'metrics' : 'rmse',
    'monotone_constraints' : [1, 0, -1]  # 1: Increasing, 0: No constraint, -1: Decreasing
}

In [ ]:
mono_model = lgb.train(


    mono_params,
    train_data,
    valid_sets=[test_data],
    num_boost_round=100,
    callbacks=[
        lgb.log_evaluation(period=50)
    ]
)

LightGBM efficiently handles missing values & sparse data without requiring imputation.
Example: Creating Sparse Features

In [ ]:
# Convert dataset to sparse matrix

X_train_sparse = scipy.sparse.csr_matrix(X_train)

X_test_sparse = scipy.sparse.csr_matrix(X_test)

In [ ]:
# Train LightGBM on sparse data

train_data_sparse = lgb.Dataset(X_train_sparse, label=y_train)

test_data_sparse = lgb.Dataset(X_test_sparse, label=y_test)

In [ ]:
# Train model

model = lgb.train(
    params, 
    train_data_sparse, 
    valid_sets=[test_data_sparse], 
    num_boost_round=100
    
    )
